In [1]:
if __name__ == "__main__":
    from pyspark.sql import SparkSession

    spark = (
        SparkSession.builder.master("local")
        .appName("Word Count")
        .config("spark.some.config.option", "some-value")
        .getOrCreate()
    )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/02 16:45:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/02 16:45:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/02 16:45:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
spark.conf.set("spark.sql.shuffle.partitions", 5)

static = spark.read.json("/data/activity-data")

In [3]:
streaming = spark\
    .readStream\
    .schema(static.schema)\
    .option("maxFilesPerTrigger", 10)\
    .json("/data/activity-data")

streaming.printSchema()

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



In [5]:
withEventTime = streaming.selectExpr(
    "*",
    "cast(cast(Creation_Time as double)/1000000000 as timestamp) as event_time")

In [8]:
from pyspark.sql.functions import window, col

withEventTime.groupBy(window(col("event_time"), "10 minutes")).count()\
                     .writeStream\
                     .queryName("pyevents_per_window")\
                     .format("memory")\
                     .outputMode("complete")\
                     .start()

24/04/02 17:00:10 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-69056d0d-887e-4527-809e-9dcc6a14ce5c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/02 17:00:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [36]:
spark.sql("SELECT * FROM pyevents_per_window").show()

+--------------------+------+
|              window| count|
+--------------------+------+
|{2015-02-24 11:50...|150773|
|{2015-02-24 13:00...|133323|
|{2015-02-23 12:30...|100853|
|{2015-02-23 10:20...| 99178|
|{2015-02-24 12:30...|125679|
|{2015-02-24 13:10...|105494|
|{2015-02-23 10:30...|100443|
|{2015-02-23 10:40...| 88681|
|{2015-02-23 13:20...|106075|
|{2015-02-22 00:40...|    35|
|{2015-02-24 11:20...|113768|
|{2015-02-24 12:20...|133623|
|{2015-02-24 14:00...|150225|
|{2015-02-24 14:10...|169064|
|{2015-02-24 13:40...|132243|
|{2015-02-24 13:50...| 96023|
|{2015-02-23 14:30...| 94669|
|{2015-02-23 13:40...|167565|
|{2015-02-24 12:00...|200133|
|{2015-02-23 12:20...|106291|
+--------------------+------+
only showing top 20 rows



In [46]:
from pyspark.sql.functions import window, col

withEventTime.groupBy(window(col("event_time"), "10 minutes"), "User").count()\
    .writeStream\
    .queryName("pyevents_per_window")\
    .format("memory")\
    .outputMode("complete")\
    .start()

24/04/02 17:03:21 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-35b370dd-32de-46e7-86fd-3c7731ce89c8. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/02 17:03:21 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


IllegalArgumentException: Cannot start query with name pyevents_per_window as a query with that name is already active in this SparkSession

In [49]:
# 30분 지연
from pyspark.sql.functions import window, col

withEventTime\
    .withWatermark("event_time", "30 minutes")\  # 워터마크
    .groupBy(window(col("event_time"), "10 minutes", "5 minutes"))\
    .count()\
    .writeStream\
    .queryName("pyevents_per_window_2")\
    .format("memory")\
    .outputMode("complete")\
    .start()

24/04/02 17:17:10 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9a915e31-3f94-436d-9946-b100d2faf839. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/02 17:17:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [96]:
spark.sql("SELECT * FROM pyevents_per_window_2").tail(5)

[Row(window=Row(start=datetime.datetime(2015, 2, 24, 15, 20), end=datetime.datetime(2015, 2, 24, 15, 30)), count=20844),
 Row(window=Row(start=datetime.datetime(2015, 2, 24, 13, 20), end=datetime.datetime(2015, 2, 24, 13, 30)), count=110594),
 Row(window=Row(start=datetime.datetime(2015, 2, 23, 14, 5), end=datetime.datetime(2015, 2, 23, 14, 15)), count=103934),
 Row(window=Row(start=datetime.datetime(2015, 2, 23, 12, 45), end=datetime.datetime(2015, 2, 23, 12, 55)), count=107713),
 Row(window=Row(start=datetime.datetime(2015, 2, 24, 15, 10), end=datetime.datetime(2015, 2, 24, 15, 20)), count=99707)]

In [101]:
query.recentProgress

NameError: name 'query' is not defined